# Implementation Playground

In [1]:
import sys
import os

sys.path.append(os.path.abspath('..'))


from simulation import params_base, state_base, compute_starting_total_length, check_d_probability

from math_spec_mapping import load_from_json

"""# For development purposes
sys.path.append(os.path.abspath('../..'))
from MSML.src.math_spec_mapping import (load_from_json)"""

from copy import deepcopy
from src import math_spec_json

ms = load_from_json(deepcopy(math_spec_json))

msi = ms.build_implementation(params_base)
state, params = msi.prepare_state_and_params(state_base, params_base,
                                             state_preperation_functions=[],
                                             parameter_preperation_functions=[])

In [6]:
spaces = []
spaces = msi.components["Add New Sample Control Action"](state, params, spaces)
print(spaces)
spaces = msi.components["Update Rolling Mean Policy"](state, params, spaces)
print(spaces)

[{'sample': 129.9812544865687}]


## Test out a Policy

In [3]:
spaces = [{'string': 'E'}]
spaces = msi.components["DUMMY Letter Count Policy"](state, params, spaces)
print(spaces)

[{'string': 'E', 'unique_length': 1, 'length': 3}]


## Combine the Two

In [4]:
spaces = []
spaces = msi.components["DUMMY Length-1 DEF Control Action"](state, params, spaces)
spaces = msi.components["DUMMY Letter Count Policy"](state, params, spaces)
print(spaces)

[{'string': 'D', 'unique_length': 1, 'length': 3}]


## Test a Mechanism

In [5]:
print("State Before:")
print(state)
spaces = [{'string': 'F', 'unique_length': 1, 'length': 3}]
msi.components["DUMMY Update Dummy Entity Mechanism"](state, params, spaces)
print()
print("State After:")
print(state)

State Before:
{'Dummy': {'Words': '', 'Total Length': None}, 'Time': 0, 'Simulation Log': [], 'Stateful Metrics': {'DUMMY Nominal Length Stateful Metric': <function dummy_metric at 0x1353db740>}, 'Metrics': {'DUMMY Multiplied Length Metric': <function dummy_multiplied_length_metric at 0x1353db9c0>}}


TypeError: unsupported operand type(s) for +=: 'NoneType' and 'int'

## Play with Stateful Metrics & Metrics

In [6]:
spaces = [{'string': 'E'}]
print(state["Metrics"]["DUMMY Multiplied Length Metric"](state, params, spaces))

3


In [7]:
print(state["Stateful Metrics"]["DUMMY Nominal Length Stateful Metric"](state, params))

1


## Run an Entire Wiring

Notice here that two new mechanisms also get run, one for incrementing time and one for logging simulation data!

In [8]:
# Reset state, params
state, params = msi.prepare_state_and_params(state_base, params_base,
                                             state_preperation_functions=[compute_starting_total_length],
                                             parameter_preperation_functions=[check_d_probability])

print("State Before:")
print(state)
spaces = []
msi.wiring["DUMMY Control Wiring"](state, params, spaces)
print()
print("State After:")
print(state)

State Before:
{'Dummy': {'Words': '', 'Total Length': 0}, 'Time': 0, 'Simulation Log': [], 'Stateful Metrics': {'DUMMY Nominal Length Stateful Metric': <function dummy_metric at 0x136f745e0>}, 'Metrics': {'DUMMY Multiplied Length Metric': <function dummy_multiplied_length_metric at 0x136f74860>}}

State After:
{'Dummy': {'Words': 'E', 'Total Length': 3}, 'Time': 1, 'Simulation Log': [{'Time': 1, 'Word': 'E', 'Length (Multiplied)': 3}], 'Stateful Metrics': {'DUMMY Nominal Length Stateful Metric': <function dummy_metric at 0x136f745e0>}, 'Metrics': {'DUMMY Multiplied Length Metric': <function dummy_multiplied_length_metric at 0x136f74860>}}
